In [21]:
import warnings
#import datasets
import os
import pandas as pd
import logging
import sklearn
import time
import random
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer,trainer_callback,DataCollatorForTokenClassification
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,classification_report
#from datasets import load_dataset, load_metric
import json
import gc
import sys
import re
from seqeval.metrics import accuracy_score, classification_report
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
def get_input_from_file(path,data_num,text_list,label_list,class_list,file_num,train_flag):
    with open(path,'r', encoding='utf-8') as f:
        standard_data = json.loads(f.read())
    print('sys.getsizeof(standard_data)',sys.getsizeof(standard_data))
    #sample_data = random.sample(standard_data,343*data_num)
    standard_data_length = len(standard_data)
    print('len of standard_data:',standard_data_length)
    
    if train_flag:
        lower_bound = file_num*data_num*1
        upper_bound = min(standard_data_length,(file_num+1)*data_num*1)
        sample_data = standard_data[lower_bound:upper_bound]
    else:
        upper_bound = min(standard_data_length,data_num*1)
        sample_data = standard_data[0:upper_bound]
        
    del standard_data
    gc.collect()
    
    for item in sample_data:
        tmp_val = []
        tmp_key = []
        shuffled_item = list(item.items())
        random.shuffle(shuffled_item)
        
        for key, val in shuffled_item:
            if key == 'class':
                class_list.append(val)
            else:
                tmp_val.append(str(val))
                tmp_key.append(key)
        text_list.append(tmp_val)
        label_list.append(tmp_key)
    return text_list,label_list,class_list
        
def get_input_from_sampling(input_size,train_flag,file_num):
    """
    input_size: the number of items to sample
    returns the input for simpletransformer
    """
    # 将抽取到的结果转化为输入格式
    text_list = []
    label_list = []
    class_list = []
    
    if train_flag:
        text_list,label_list,class_list = get_input_from_file('preprocess/standard_cap.json',input_size,text_list,label_list,class_list,file_num,train_flag)
        text_list,label_list,class_list = get_input_from_file('preprocess/standard_res.json',input_size,text_list,label_list,class_list,file_num,train_flag)
    else:
        text_list,label_list,class_list = get_input_from_file('preprocess/standard_cap_test.json',input_size,text_list,label_list,class_list,file_num,train_flag)
        text_list,label_list,class_list = get_input_from_file('preprocess/standard_res_test.json',input_size,text_list,label_list,class_list,file_num,train_flag)

    #to shuffle input
    res=[]
    for i in range(len(label_list)):
        res.append((text_list[i],label_list[i],class_list[i]))
    random.shuffle(res)
    for i in range(len(label_list)):
        text_list[i]=res[i][0]
        label_list[i]=res[i][1]
        class_list[i]=res[i][2]
        
        
    print('query数量：',len(text_list))
    print('label数量：',len(label_list))
    print('class数量：',len(class_list))
    
    print('sys.getsizeof(text_list)',sys.getsizeof(text_list))
    print('sys.getsizeof(label_list)',sys.getsizeof(label_list))
    
    return text_list,label_list,class_list

# a,b,c=get_input_from_sampling(500,True,1)
# for i in range(len(b)):
#     print((a[i],b[i],c[i]),'\n')

sys.getsizeof(standard_data) 9024
len of standard_data: 1000
sys.getsizeof(standard_data) 7056
len of standard_data: 849
query数量： 849
label数量： 849
class数量： 849
sys.getsizeof(text_list) 7056
sys.getsizeof(label_list) 7056
(['10000mV', '20%', '125℃', '100000n', '2611'], ['RatedDCVoltageURdc', 'PositiveTolerance', 'OperatingTemperatureMax', 'Capacitance', 'SizeCode'], 'Capacitors') 

(['贴片电容', 'THROUGH HOLE MOUNT', '1096uF'], ['input class', 'MountingFeature', 'Capacitance'], 'Capacitors') 

(['THROUGH HOLE MOUNT', '6237u', '50000mV'], ['MountingFeature', 'Capacitance', 'RatedDCVoltageURdc'], 'Capacitors') 

(['3478p', '8246kV', '20%'], ['Capacitance', 'RatedDCVoltageURdc', 'PositiveTolerance'], 'Capacitors') 

(['THROUGH HOLE MOUNT', '6588kV', '2200000000pF'], ['MountingFeature', 'RatedDCVoltageURdc', 'Capacitance'], 'Capacitors') 

(['6.3V', 'CAP', '6913uF', '105℃', '20%', 'ALUMINUM (WET)', 'THROUGH HOLE MOUNT'], ['RatedDCVoltageURdc', 'input class', 'Capacitance', 'OperatingTemperature


(['0805', '42%', '6513kΩ', '-100,100', 'SMT'], ['SizeCode', 'Tolerance', 'Resistance', 'TemperatureCoefficient', 'PackageStyle'], 'Resistors') 

(['ALUMINUM (WET)', '400V', '20%', '3434pF'], ['DielectricMaterial', 'RatedDCVoltageURdc', 'PositiveTolerance', 'Capacitance'], 'Capacitors') 

(['RES', 'R0502', '2%', '7413MΩ'], ['input class', 'SizeCode', 'Tolerance', 'Resistance'], 'Resistors') 

(['6.8nF', '5%', 'cap', 'POLYESTER', '317kV'], ['Capacitance', 'PositiveTolerance', 'input class', 'DielectricMaterial', 'RatedDCVoltageURdc'], 'Capacitors') 

(['6691u', '4.7u', '1411', 'TANTALUM (DRY/SOLID)', 'Capacitors', '-55℃'], ['RatedDCVoltageURdc', 'Capacitance', 'SizeCode', 'DielectricMaterial', 'input class', 'OperatingTemperatureMin'], 'Capacitors') 

(['4326p', '2917', '125℃'], ['Capacitance', 'SizeCode', 'OperatingTemperatureMax'], 'Capacitors') 

(['6800000pF', '20%', '2525kV'], ['Capacitance', 'PositiveTolerance', 'RatedDCVoltageURdc'], 'Capacitors') 

(['10000m', '100uF', '2915', '

In [3]:
# #保存列表，每行一个元素
# with open('input_example.txt','w',encoding='utf-8') as f:
#     d=[]
#     for i in range(len(b)):
#         d.append(c[i]+'       '+str(b[i])+'       '+str(a[i]))
#     f.write('\n'.join(d))

In [4]:
def tokenize_and_align_labels(text,labels_before_split,tokenizer):
    tokenized_inputs = tokenizer(text, is_split_into_words=True,add_special_tokens=False,
                        padding=True,truncation=False,return_tensors="pt")
    labels = []
    for i, label in enumerate(labels_before_split):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append('-100')
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

#    tokenized_inputs["labels"] = labels
    return tokenized_inputs,labels

In [8]:
def show_tokenizer_result(text,labels_before_split,tokenizer):
    tokenized_inputs,labels = tokenize_and_align_labels(text,labels_before_split,tokenizer)
    for i in range(len(labels_before_split)):
        print('text:',text[i])
        print('tokenized result:',tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][i]))
        #print('tokenized_inputs:',tokenized_inputs["input_ids"][i])
        
        print('labels_before_split:',labels_before_split[i])
        print('labels:',labels[i])
        print('\n%%%%%%%%%%%%%%%%%%%%%\n')

#show_tokenizer_result(a[:10],b[:10],tokenizer)

In [9]:
# 读文件里面的数据转化为二维列表
def read_list(filename):
    file1 = open(filename+".txt", "r")
    list_row =file1.readlines()
    list_source = []
    for i in range(len(list_row)):
        column_list = list_row[i].strip().split("\t")  # 每一行split后是一个列表
        list_source.append(column_list)                # 在末尾追加到list_source
    file1.close()
    return list_source

#保存二维列表到文件
def save_list(list1,filename):
    file2 = open(filename + '.txt', 'w')
    for i in range(len(list1)):
        for j in range(len(list1[i])):
            file2.write(str(list1[i][j]))              # write函数不能写int类型的参数，所以使用str()转化
            file2.write('\t')                          # 相当于Tab一下，换一个单元格
        file2.write('\n')                              # 写完一行立马换行
    file2.close()

In [10]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [72]:
# #注意最开始的label处理部分，已经把没意义的赋值为-100了
# cap_attrs = {'Capacitance':1,'SizeCode':2,'RatedDCVoltageURdc':3,'PositiveTolerance':4,'NegativeTolerance':5,'TemperatureCharacteristicsCode':6,'MfrPartNumber':7,'input class':8}
# res_attrs = {'Resistance':11,'SizeCode':12,'WorkingVoltage':13,'Tolerance':14,'RatedPowerDissipationP':15,'MfrPartNumber':10,'input class':9}
# all_class_list = {'Capacitors':cap_attrs,'Resistors':res_attrs}

# cap_attrs_dict = dict(zip(cap_attrs.values(), cap_attrs.keys()))
# res_attrs_dict = dict(zip(res_attrs.values(), res_attrs.keys()))
# all_attrs_dict ={**cap_attrs_dict,**res_attrs_dict}

def attrEncoder(all_class_list,item_class,attr):
    if item_class in all_class_list and attr in all_class_list[item_class]:
        return all_class_list[item_class][attr]
    if attr == '-100':
        return -100
    return 0
    
def attrDecoder(attr):
    if attr==0 or attr==-100:
        return 'others'
    else:
        global all_attrs_dict
        return all_attrs_dict[attr]

#attrDecoder(5)

In [18]:
# train_encodings = tokenizer(train_dataset, is_split_into_words=True,add_special_tokens=False,padding=True,truncation=True)
# for i in range(len(train_encodings)):
#     print('tokenized result:',tokenizer.convert_ids_to_tokens(train_encodings["input_ids"][i]))


In [86]:
def train_input_process(model_checkpoint,train_size,tokenizer,file_num):
    """
    """
    files= os.listdir('./inputData')

#     with open('running_output.txt','w') as f:
#         f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+'\n')
    
    #准备训练数据
    print(f'train dataset{file_num}\n')
    if f'train_encodings{file_num}.pt' in files and f'train_labels{file_num}.txt' in files and f'train_class{file_num}.csv' in files:
        print('read train dataset\n')
        train_encodings = torch.load(f'./inputData/train_encodings{file_num}.pt')
        train_labels = read_list(f'./inputData/train_labels{file_num}')
        train_class = pd.read_csv(f'./inputData/train_class{file_num}.csv',index_col=0)
        train_class = train_class['0'].to_list()
    else:
        print('generate train dataset\n')
        train_dataset,train_labels,train_class = get_input_from_sampling(train_size,train_flag=True,file_num=file_num)
        #train_encodings = tokenizer(train_dataset, is_split_into_words=True,add_special_tokens=False,
#                        padding=True,truncation=False,return_tensors="pt")
        train_encodings,train_labels = tokenize_and_align_labels(train_dataset,train_labels,tokenizer)

        pd.DataFrame(train_class).to_csv(f'./inputData/train_class{file_num}.csv')
        save_list(train_labels,f'./inputData/train_labels{file_num}')
        torch.save(train_encodings, f'./inputData/train_encodings{file_num}.pt')
        print('train dataset saved')
        
    # encode the labels
    global all_class_list
    train_labels_encoded = []
    for i in range(len(train_labels)):
#        train_labels_encoded[i] = list(map(lambda x,y,z:attrEncoder(x,y,z),all_class_list,train_class[i],train_labels[i]))
        train_labels_encoded.append([attrEncoder(all_class_list,train_class[i],train_label) for train_label in train_labels[i]])
    
    print('training dataset is ok\n')
    return train_encodings,train_labels_encoded,train_class

def test_input_process(model_checkpoint,test_size,tokenizer):
    """
    """
    print('generate test dataset\n')
    test_dataset,test_labels,test_class = get_input_from_sampling(test_size,train_flag=False,file_num=1)
    test_encodings,test_labels = tokenize_and_align_labels(test_dataset,test_labels,tokenizer)

    # encode the labels
    global all_class_list
    test_labels_encoded = []
    for i in range(len(test_labels)):
        test_labels_encoded.append([attrEncoder(all_class_list,test_class[i],test_label) for test_label in test_labels[i]]) 
                  
    print('testing dataset is ok\n')
    return test_encodings,test_labels_encoded,test_class,test_dataset[:1000],test_labels

#train_data,train_labels_encoded,train_class = train_input_process(model_checkpoint,3,tokenizer,1)
#te_data,te_labels_encoded,te_class,test_sample,test_labels = test_input_process(model_checkpoint,10,tokenizer)

In [13]:
class processDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
def compute(predictions, references, suffix=False):
    report = classification_report(y_true=references, y_pred=predictions, suffix=suffix, output_dict=True)
    report.pop("macro avg")
    report.pop("weighted avg")
    overall_score = report.pop("micro avg")

    scores = {
        type_name: {
            "precision": score["precision"],
            "recall": score["recall"],
            "f1": score["f1-score"],
            "number": score["support"],
        }
        for type_name, score in report.items()
    }
    scores["overall_precision"] = overall_score["precision"]
    scores["overall_recall"] = overall_score["recall"]
    scores["overall_f1"] = overall_score["f1-score"]
    scores["overall_accuracy"] = accuracy_score(y_true=references, y_pred=predictions)

    return scores

In [60]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Remove ignored index (special tokens)
    global all_class_list
    true_predictions = [
        [attrDecoder(p) for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    true_labels = [
        [attrDecoder(l) for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    
    results = compute(predictions=true_predictions, references=true_labels)
    #print(results)
    
    #对result进行处理，以写入文件
    for k,v in results.items():
        if isinstance(v,dict):
            v['number']=str(v['number'])
        
#     precision, recall, f1, _ = precision_recall_fscore_support(true_labels, true_predictions, average='micro',zero_division=1)
#     acc = accuracy_score(labels, preds)
    
#     print('compute_metrics:',len(labels))
#     class_result = {'accuracy': acc,
#                 'f1': f1,
#                 'precision': precision,
#                 'recall': recall}
    
#     with open('class_classification_report.json','a',errors='ignore') as f: 
#         json.dump(class_report,f,ensure_ascii=False, indent = 4) 

    global write_result
    print('write_result',write_result)
    if write_result:
#         file = open('class_classification_report.txt', 'w')
#         for k,v in dict.items():
#             file.write(k + ' ')
#             for k1, v1 in v.items():
#                 file.write(k1 + ' ' + str(v1) + ' ')
#             file.write(' \n')
#         file.close()
#         class_preds = [attrDecoder(item) for item in true_predictions]
#         class_labels = [attrDecoder(item) for item in true_labels]
        #class_report = classification_report(class_labels, class_preds,output_dict=True)
        #class_report = classification_report(class_labels, class_preds)
#         with open('class_classification_report.txt','a') as f: 
#             f.write(str(results))

        with open('class_classification_report.json','a',errors='ignore') as f: 
            json.dump(results,f,ensure_ascii=False, indent = 4) 
            
#     with open('class_running_output.txt','w') as f:
#         f.write('class_result:'+str(class_result)+'\n')

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]}

In [15]:
# write_result = True
# #model_checkpoint = "albert-base-v2"
# model_checkpoint = "prajjwal1/bert-tiny"
# # model_checkpoint = r'C:\Users\coldkiller\Desktop\supplyframe\checkpoint-3500'
# gpu_available = torch.cuda.is_available()
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# #先识别cap一个类的
# model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=16)
# #data_collator = DataCollatorForTokenClassification(tokenizer,padding=True)
# #metric = load_metric("seqeval")

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from t

In [16]:
# train_data,train_labels_encoded,train_class = train_input_process(model_checkpoint,10,tokenizer,1)
# test_data,test_labels_encoded,test_class,test_sample,test_labels = test_input_process(model_checkpoint,5,tokenizer)
# train_dataset = processDataset(train_data, train_labels_encoded)
# test_dataset = processDataset(test_data, test_labels_encoded)

train dataset1

read train dataset

training dataset is ok

generate test dataset

sys.getsizeof(standard_data) 9024
len of standard_data: 1000
sys.getsizeof(standard_data) 7056
len of standard_data: 849
query数量： 10
label数量： 10
class数量： 10
sys.getsizeof(text_list) 192
sys.getsizeof(label_list) 192
testing dataset is ok



In [61]:
# training_args = TrainingArguments(
#     output_dir='./models',
#     do_train = True,
#     learning_rate=1e-5,
#     num_train_epochs=1,
#     logging_steps=1,
#     no_cuda= not gpu_available,
#     seed=1024,
#     load_best_model_at_end=True,
#     disable_tqdm=True)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset
# )
# trainer.train()

{'loss': 2.649495840072632, 'learning_rate': 6.666666666666667e-06, 'epoch': 0.3333333333333333}
{'loss': 2.778709888458252, 'learning_rate': 3.3333333333333333e-06, 'epoch': 0.6666666666666666}
{'loss': 2.8111603260040283, 'learning_rate': 0.0, 'epoch': 1.0}
{'epoch': 1.0}


TrainOutput(global_step=3, training_loss=2.746455351511637)

In [62]:
# with open('class_classification_report.txt','a') as f: 
#     f.write('~~~~~~~~~~training result:~~~~~~~~~~\n')
# train_result = trainer.predict(train_dataset).metrics
# print('train_result:  ',train_result)

# with open('class_classification_report.txt','a') as f: 
#     f.write('~~~~~~~~~~testing result:~~~~~~~~~~\n')
# test_result = trainer.predict(test_dataset)
# print('test_result:  ',test_result.metrics)

{'Capacitance': {'precision': 0.12612612612612611, 'recall': 0.4117647058823529, 'f1': 0.19310344827586207, 'number': 34}, 'NegativeTolerance': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}, 'PositiveTolerance': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}, 'RatedDCVoltageURdc': {'precision': 0.21428571428571427, 'recall': 0.17647058823529413, 'f1': 0.1935483870967742, 'number': 34}, 'RatedPowerDissipationP': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 12}, 'Resistance': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 36}, 'SizeCode': {'precision': 0.04411764705882353, 'recall': 0.125, 'f1': 0.06521739130434782, 'number': 24}, 'TemperatureCharacteristicsCode': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'Tolerance': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 19}, 'input class': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 12}, 'others': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 42}, 'overall_pr

In [87]:
def train(num_epoch=10,logging_steps=10,file_num=10,train_size=150,test_size=30):

    set_seed(1024)
    global write_result
    write_result = True
    model_checkpoint = "albert-base-v2"
   # model_checkpoint = "prajjwal1/bert-tiny"
    # model_checkpoint = r'C:\Users\coldkiller\Desktop\supplyframe\checkpoint-3500'
    
    gpu_available = torch.cuda.is_available()
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    
    print('torch.cuda.is_available()',gpu_available)
    
    #两个类放在一起
    model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=16)
    training_args = TrainingArguments(
        output_dir='./models',
        dataloader_num_workers=7,
        do_train = True,
#        do_eval = True,
#        evaluation_strategy = 'steps',
        learning_rate=1e-5,
        weight_decay=0.01,
        adam_beta1=0.9,
        adam_beta2=0.999,
        adam_epsilon=1e-8,
        num_train_epochs=num_epoch,
        logging_steps=logging_steps,
        save_steps=10000,
        no_cuda= not gpu_available,
        seed=1024,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        warmup_steps=20,
        logging_dir='./logs',
        load_best_model_at_end=True,
#        metric_for_best_model = 'eval_loss',
#        greater_is_better = False,
#        save_total_limit=5,
        disable_tqdm=True)
    
    # read real data
#     with open(r'preprocess/description_with_label.json', 'r', errors='ignore', encoding='utf-8') as f:
#         js = f.read()
#         real_data = json.loads(js, strict=False)

#     real_input=[]
#     real_labels=[]
#     for i in range(len(real_data)):
#         if 'class' in real_data[i]:
#             real_input.append(real_data[i]['description'])
#             real_labels.append(attrEncoder(real_data[i]['class']))

#     print('real_input',len(real_input))
#     real_encodings = tokenizer(real_input,padding=True,truncation=False)
#     real_dataset = processDataset(real_encodings,real_labels)

    # training loop
    total_time = 0
    for j in range(num_epoch):
        print(f'\n\n%%%%%%%%%%%%%%epoch{j}%%%%%%%%%%%%%%\n\n')
        for i in range(file_num):
            print(f'\n\nbegin generate train_dataset{i}\n\n')
            train_data,train_labels_encoded,train_class = train_input_process(model_checkpoint,train_size,tokenizer,i)
            train_dataset = processDataset(train_data, train_labels_encoded)

    #         if i>0:
    #             trainer.args.learning_rate = tmp_learning_rate
            
            trainer = Trainer(
                model=model,
                args=training_args,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics,
                train_dataset=train_dataset
            )
            # trainer.add_callback(trainer_callback.EarlyStoppingCallback(early_stopping_patience=500,early_stopping_threshold=0.0001))
            
            trainer.args.output_dir=f'./models/model{j}'
            # Train the model
            print('Training begins\n')
            start = time.time()
            trainer.train()
            end = time.time()
            print(f"training time: {end - start}")
            total_time+=(end-start)

            #为下一批训练数据做初始化
            trainer.args.warmup_steps=0
            #tmp_learning_rate = trainer.args.learning_rate
            gc.collect()
    
    # 保存模型
    trainer.save_model()
        
#         with open('class_classification_report.txt','a') as f: 
#             f.write(f'%%%%%%%%%%epoch {j}%%%%%%%%%%%\n')
#         real_result = trainer.predict(real_dataset)
#         print('real_result:  ',real_result.metrics)
#         with open('validation_report.txt','a') as f: 
#             f.write(f'epoch {j}:{real_result.metrics}\n')
#     with open('albert_structure.txt','w') as f:
#         f.write(str(trainer.model))

    print('\n\ntotal_training_time:',total_time)
    # generate test data
    test_data,test_labels_encoded,test_class,sample_data,test_labels = test_input_process(model_checkpoint,test_size,tokenizer)
    test_dataset = processDataset(test_data, test_labels_encoded)
    
    # show result
#     global write_result
#     write_result = True
    with open('class_classification_report.txt','a') as f: 
        f.write('~~~~~~~~~~training result:~~~~~~~~~~\n')
    train_result = trainer.predict(train_dataset).metrics
    print('train_result:  ',train_result)
    
    with open('class_classification_report.txt','a') as f: 
        f.write('~~~~~~~~~~testing result:~~~~~~~~~~\n')
    test_result = trainer.predict(test_dataset)
    print('test_result:  ',test_result.metrics)
    
#     with open('class_classification_report.txt','a') as f: 
#         f.write('~~~~~~~~~~validation result:~~~~~~~~~~\n')
#     real_result = trainer.predict(real_dataset)
#     print('real_result:  ',real_result.metrics)
    
    # save result
    test_preds = [[attrDecoder(p) for (p, l) in zip(prediction, label) if l != -100]
                    for prediction, label in zip(test_result.predictions.argmax(-1)[:1000], test_labels_encoded[:1000])]
    test_labels = [[attrDecoder(l) for (p, l) in zip(prediction, label) if l != -100]
                    for prediction, label in zip(test_result.predictions.argmax(-1)[:1000], test_labels_encoded[:1000])]

    # test_preds = [[attrDecoder(attr) for attr in item] for item in test_result.predictions.argmax(-1)[:3]]
    # test_labels = [[attrDecoder(attr) for attr in item] for item in test_labels_encoded[:3]]
    test_res = pd.DataFrame({"description":sample_data,"true_labels":test_labels,"predicted_labels":test_preds})
    test_res.to_csv('test_description_result.csv',encoding='utf_8_sig')
    
#     real_preds = [attrDecoder(item) for item in real_result.predictions.argmax(-1)]
#     real_labels = [attrDecoder(item) for item in real_labels]
#     real_res = pd.DataFrame({"description":real_input,"true_labels":real_labels,"predicted_labels":real_preds})
#     real_res.to_csv('real_description_result.csv',encoding='utf_8_sig')
    
#     with open('running_output.txt','a') as f:
#         f.write(f"training time: {end - start}"+'\n')
#         f.write('train_dataset'+str(train_result)+'\n')        
#         f.write('test_dataset'+str(test_result)+'\n')
        
#     with open('real_description_output.json','w',errors='ignore') as f: 
#         json.dump(class_result,f,ensure_ascii=False, indent = 4) 

In [88]:
if __name__=="__main__":
    #注意最开始的label处理部分，已经把没意义的赋值为-100了
    cap_attrs = {'Capacitance':1,'SizeCode':2,'RatedDCVoltageURdc':3,'PositiveTolerance':4,'NegativeTolerance':5,'TemperatureCharacteristicsCode':6,'MfrPartNumber':7,'input class':8}
    res_attrs = {'Resistance':11,'SizeCode':12,'WorkingVoltage':13,'Tolerance':14,'RatedPowerDissipationP':15,'MfrPartNumber':10,'input class':9}
    all_class_list = {'Capacitors':cap_attrs,'Resistors':res_attrs}

    cap_attrs_dict = dict(zip(cap_attrs.values(), cap_attrs.keys()))
    res_attrs_dict = dict(zip(res_attrs.values(), res_attrs.keys()))
    all_attrs_dict ={**cap_attrs_dict,**res_attrs_dict}
    
    write_result = True
    train(num_epoch=100,logging_steps=100,file_num=9,train_size=110000,test_size=30000)


torch.cuda.is_available() False


Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from t



%%%%%%%%%%%%%%epoch0%%%%%%%%%%%%%%




begin generate train_dataset0


train dataset0

read train dataset

training dataset is ok

Training begins

{'loss': 2.7033333778381348, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.0}
{'epoch': 1.0}
training time: 0.14960050582885742


begin generate train_dataset1


train dataset1

read train dataset

training dataset is ok

Training begins

{'loss': 2.751540184020996, 'learning_rate': 0.0, 'epoch': 1.0}
{'epoch': 1.0}
training time: 0.14960050582885742


%%%%%%%%%%%%%%epoch1%%%%%%%%%%%%%%




begin generate train_dataset0


train dataset0

read train dataset

training dataset is ok

Training begins

{'loss': 2.6998131275177, 'learning_rate': 0.0, 'epoch': 1.0}
{'epoch': 1.0}
training time: 0.15259170532226562


begin generate train_dataset1


train dataset1

read train dataset

training dataset is ok

Training begins

{'loss': 2.7427046298980713, 'learning_rate': 0.0, 'epoch': 1.0}
{'epoch': 1.0}
training time: 0.1515944004058838


tot

In [ ]:
#train(num_epoch=2,logging_steps=1,file_num=30,train_size=100,test_size=300)


In [ ]:
#改比例，训练数据数量，epoch,dataloader_num_workers,模型,sample的数量

In [ ]:
def show_prediction_result(model):
    """
    model: the trained model
    returns the predicted result
    """
    rs_list = get_rs_list_of_all_category()
    rs = random.choice(rs_list)
    item = rs.random_sampling()
    truth_result = pd.DataFrame.from_dict(item,orient='index').reset_index()
    truth_result.columns = ['output_truth','input']
    truth_result['input'] = truth_result['input']
    print(truth_result)
    input_sentence = list(item.values())
    input_sentence = [str(x) for x in input_sentence]
    predictions, raw_outputs = model.predict([input_sentence],split_on_space= False )
    
    pred_dict={}
    for item in predictions[0]:
        pred_dict.update(item)
    pred_result = pd.DataFrame.from_dict(pred_dict,orient='index').reset_index()
    pred_result.columns = ['input','output_pred']
    print(pred_result)
    show_result = pd.merge(pred_result,truth_result,how='right',on='input')
    return show_result
    show_prediction_result(model)